# ***TRANSFORMATION***

Data transformation involves various techniques that can be applied based on the specific requirements. Some common examples of data transformations include:

1. Creating new columns


2. Normalizing data


3. Merging datasets


4. Performing data aggregation

In [ ]:
import pandas as pd

In [ ]:
booking=pd.read_csv("/content/fact_bookings.csv")
agg_booking=pd.read_csv("/content/fact_aggregated_bookings.csv")
room=pd.read_csv("/content/dim_rooms.csv")
hotel=pd.read_csv("/content/dim_hotel.csv")

In [ ]:
agg_booking.head(5)

,property_id,check_in_date,room_category,successful_bookings,capacity
0,16559,1-May-22,RT1,25,30.0
1,19562,1-May-22,RT1,28,30.0
2,19563,1-May-22,RT1,23,30.0
3,17558,1-May-22,RT1,30,19.0
4,16558,1-May-22,RT1,18,19.0


In [ ]:
new_col = agg_booking.apply(lambda row: row['successful_bookings']/row['capacity'], axis=1)
agg_booking = agg_booking.assign(occ_pct=new_col.values)
agg_booking.head(3)

,property_id,check_in_date,room_category,successful_bookings,capacity,occ_pct
0,16559,1-May-22,RT1,25,30.0,0.833333
1,19562,1-May-22,RT1,28,30.0,0.933333
2,19563,1-May-22,RT1,23,30.0,0.766667


## Convert it into a percentage value

In [ ]:
agg_booking['occ_pct'] = agg_booking['occ_pct'].apply(lambda x: round(x*100, 2))
agg_booking.head(5)

,property_id,check_in_date,room_category,successful_bookings,capacity,occ_pct
0,16559,1-May-22,RT1,25,30.0,83.33
1,19562,1-May-22,RT1,28,30.0,93.33
2,19563,1-May-22,RT1,23,30.0,76.67
3,17558,1-May-22,RT1,30,19.0,157.89
4,16558,1-May-22,RT1,18,19.0,94.74


In [ ]:
merge=pd.merge(agg_booking,room,left_on="room_category",right_on="room_id").head(5)
merge


,property_id,check_in_date,room_category,successful_bookings,capacity,occ_pct,room_id,room_class
0,16559,1-May-22,RT1,25,30.0,83.33,RT1,Standard
1,19562,1-May-22,RT1,28,30.0,93.33,RT1,Standard
2,19563,1-May-22,RT1,23,30.0,76.67,RT1,Standard
3,17558,1-May-22,RT1,30,19.0,157.89,RT1,Standard
4,16558,1-May-22,RT1,18,19.0,94.74,RT1,Standard


In [ ]:
booking.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134590 entries, 0 to 134589
Data columns (total 12 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   booking_id         134590 non-null  object 
 1   property_id        134590 non-null  int64  
 2   booking_date       134590 non-null  object 
 3   check_in_date      134590 non-null  object 
 4   checkout_date      134590 non-null  object 
 5   no_guests          134587 non-null  float64
 6   room_category      134590 non-null  object 
 7   booking_platform   134590 non-null  object 
 8   ratings_given      56683 non-null   float64
 9   booking_status     134590 non-null  object 
 10  revenue_generated  134590 non-null  int64  
 11  revenue_realized   134590 non-null  int64  
dtypes: float64(2), int64(3), object(7)
memory usage: 12.3+ MB


# ***INSIGHTS GENERATION***

# *1. What is the occupancy rate for each room type?*

In [ ]:
  agg_booking.groupby("room_category")["occ_pct"].mean()

,occ_pct
room_category,
RT1,58.224247
RT2,58.040278
RT3,58.028213
RT4,59.300461


I didn’t fully understand the occupancy rate based on room categories like RT1, RT2, etc. To make it clearer, I want to merge the room categories into broader classes such as Standard, Premium, and Elite.

In [ ]:
merge_room = pd.merge(agg_booking, room, left_on="room_category", right_on="room_id")
merge_room.head(5)

,property_id,check_in_date,room_category,successful_bookings,capacity,occ_pct,room_id,room_class
0,16559,1-May-22,RT1,25,30.0,83.33,RT1,Standard
1,19562,1-May-22,RT1,28,30.0,93.33,RT1,Standard
2,19563,1-May-22,RT1,23,30.0,76.67,RT1,Standard
3,17558,1-May-22,RT1,30,19.0,157.89,RT1,Standard
4,16558,1-May-22,RT1,18,19.0,94.74,RT1,Standard


In [ ]:
merge_room.drop("room_id",axis=1, inplace=True)
merge_room.head(4)

,property_id,check_in_date,room_category,successful_bookings,capacity,occ_pct,room_class
0,16559,1-May-22,RT1,25,30.0,83.33,Standard
1,19562,1-May-22,RT1,28,30.0,93.33,Standard
2,19563,1-May-22,RT1,23,30.0,76.67,Standard
3,17558,1-May-22,RT1,30,19.0,157.89,Standard


In [ ]:
merge_room.groupby("room_class")["occ_pct"].mean()

,occ_pct
room_class,
Elite,58.040278
Premium,58.028213
Presidential,59.300461
Standard,58.224247


# *2. Print an average occupancy rate per city?*

In [ ]:
hotel.head(5)

,property_id,property_name,category,city
0,16558,Radison Elite,Luxury,Delhi
1,16559,Radison standard,Luxury,Mumbai
2,16560,Radison City,Business,Delhi
3,16561,Radison Blu,Luxury,Delhi
4,16562,Radison Bay,Luxury,Delhi


In [ ]:
merge_hotel= pd.merge(agg_booking , hotel, on="property_id")
merge_hotel.head(3)

,property_id,check_in_date,room_category,successful_bookings,capacity,occ_pct,property_name,category,city
0,16559,1-May-22,RT1,25,30.0,83.33,Radison standard,Luxury,Mumbai
1,19562,1-May-22,RT1,28,30.0,93.33,Radison Bay,Luxury,Bangalore
2,19563,1-May-22,RT1,23,30.0,76.67,Radison palace,Business,Bangalore


In [ ]:
merge_hotel.groupby("city")["occ_pct"].mean()

,occ_pct
city,
Bangalore,56.594207
Delhi,61.606467
Hyderabad,58.144651
Mumbai,57.936305


# *3. What is the average customer rating by room type?*

In [ ]:
booking.head(5)

,booking_id,property_id,booking_date,check_in_date,checkout_date,no_guests,room_category,booking_platform,ratings_given,booking_status,revenue_generated,revenue_realized
0,May012216558RT11,16558,27-04-22,1/5/2022,2/5/2022,-3.0,RT1,direct online,1.0,Checked Out,10010,10010
1,May012216558RT12,16558,30-04-22,1/5/2022,2/5/2022,2.0,RT1,others,NaN,Cancelled,9100,3640
2,May012216558RT13,16558,28-04-22,1/5/2022,4/5/2022,2.0,RT1,logtrip,5.0,Checked Out,9100000,9100
3,May012216558RT14,16558,28-04-22,1/5/2022,2/5/2022,-2.0,RT1,others,NaN,Cancelled,9100,3640
4,May012216558RT15,16558,27-04-22,1/5/2022,2/5/2022,4.0,RT1,direct online,5.0,Checked Out,10920,10920


In [ ]:
booking.groupby("room_category")["ratings_given"].mean().reset_index()

,room_category,ratings_given
0,RT1,3.631829
1,RT2,3.602902
2,RT3,3.592317
3,RT4,3.686919


# *4. Which room type generates the most revenue realized?*

In [ ]:
booking.groupby("room_category")["revenue_realized"].mean().reset_index()

,room_category,revenue_realized
0,RT1,8052.356422
1,RT2,11317.467003
2,RT3,15120.275600
3,RT4,23440.103652


# *5. Which room type generates the most revenue generated?*

In [ ]:
booking.groupby("room_category")["revenue_generated"].mean().reset_index()

,room_category,revenue_generated
0,RT1,9744.187822
1,RT2,13766.709625
2,RT3,18717.832886
3,RT4,27465.751260
